# Gradient Descent, the Normal Equation, Learning Rates, and why all of this matters for Linear Regression

Linear regression is typically used as the hello world for data science. It's one of the oldest machine learning models (designed far before machines could learn) but, despite being so old, it's far from being the simplest model. Understanding linear regression is also the first step toward understanding more sophisticated models like logistic regression, support vector machines, and even deep learning models.

Below I'll get into some of the nitty-gritty of linear regression so that those reading can have a deeper understanding of how the model works and, if you choose, create your own implementation of it.

I don't see any point in waiting, so, let's dive into the code and do some necessary imports.

We'll use `numpy` for speed, and `minmax_scale`, `train_test_split`, and `r2_score` from `sklearn` for convenience and `LinearRegression` from `sklearn` to test some of our implementations. We've also imported `time` and `altair` to compare the time complexity of one of our implementations against `sklearn`'s.

In [1]:
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from time import time
import altair as alt

Rather than direct everyone to a potentially broken link, feel free to test any implementations using the code below to generate data.

In [2]:
def create_data(n_feats=8, n_obs=1000):
    features = ['f' + str(x) for x in range(n_feats)]
    target = abs(np.random.normal(1000, 250, n_obs))
    feat_vals = []
    for feature in features:
        feat_vals.append([np.random.normal(
                            abs(np.random.normal(target[i],
                            abs(np.random.normal(100, 30)))))
                            for i in range(n_obs)])
    X = np.array(feat_vals).T
    return target, X

In [106]:
target, X = create_data(n_feats=10, n_obs=10)
X_scaled = minmax_scale(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, target,
                                                    test_size=0.33,
                                                    random_state=42)

## The Line of Best Fit

We can't talk about machine learning models like Linear Regression without first quickly reviewing where these models come from.  

The great grandfather of linear regression is the line of best fit. You use it when you have a single input variable and a single output variable. That's when it works.

Given $n \times 1$ explanatory and response variables, we can calculate the slope and intercept of the line of best fit.

In [97]:
x_train = X_train[:, 0]
x_test = X_test[:, 0]

x_mean = x_train.mean()
y_mean = y_train.mean()
m = ((x_train - x_mean) * (y_train - y_mean)).sum() / ((x_train - x_mean)**2).sum()
b = y_mean - m * x_mean
preds = m * x_test + b
1 - ((y_test - preds)**2).sum() / ((y_test - y_mean)**2).sum()        

0.8706799041962858

This can be done without iteration. We simply find the slope of the line (`m`) and then use that while calculating our intercept (`b`).  

We can see that the prediction is (almost) the same as when we use sklearn's LinearRegression

In [98]:
from sklearn.linear_model import LinearRegression

LinearRegression().fit(x_train.reshape(-1, 1), y_train).score(x_test.reshape(-1, 1), y_test)

0.8622231033771592

## Normal Equation Method
The line of best fit works when we have a single explanatory variable. What do we do when we have multiple explanatory variables? For that we take a huge leap forward in complexity and use the normal equation; the equation that we get from differentiating the residual sum of sqaures equation.

The derivative of RSS deserves it's entire own blog/video, fortunately it's already been done a million times over. Search for your own explanation or just look at this [video](https://www.youtube.com/watch?v=K_EH2abOp00) or at [wiki](https://en.wikipedia.org/wiki/Proofs_involving_ordinary_least_squares) for both the matrix and summation notation.

Ultimately, when we differentiate the residual sum of squares equation:  
$RSS={\bigl \|}\mathbf {y} -\mathbf {X}{\beta }{\bigl \|}^2$  
we get:  
$(\mathbf {X} ^{\rm {T}}\mathbf {X} )^{-1}\mathbf {X} ^{\rm {T}}\mathbf {y} =0$  

This is the equation that we'll use in our next example.

In [99]:
class NormalEquation:
    
    def __init__(self):
        pass
    
    def __repr__(self):
        return "OLS()"
    
    def fit(self, X_train, y_train):
        # Our new column that we'll call intercept
        # We make it based on the shape of the first column in our training data
        intercept = np.ones_like(X_train[:, 0])
        
        # We add our new column to our training data
        X_train_int = np.insert(X_train, 0, intercept, axis=1)
        
        # Use the normal equation
        coefs = np.linalg.inv(X_train_int.T @ X_train_int) @ X_train_int.T @ y_train
        
        # Save our coefficients for later use
        self.coefs_ = coefs
        return self
    
    def predict(self, X_test):
        # Again, we create a new column for the test data
        intercept = np.ones_like(X_test[:, 0])
        X_test_int = np.insert(X_test, 0, intercept, axis=1)
        preds = X_test_int @ self.coefs_
        return preds
    
    def score(self, X_test, y_test):
        # Again, we create a new column for the test data
        intercept = np.ones_like(X_test[:, 0])
        X_test_int = np.insert(X_test, 0, intercept, axis=1)
        preds = X_test_int @ self.coefs_
        
        # Here we're simply calculating r-squared
        score = 1 - ((y_test - preds)**2).sum() / ((y_test - y_test.mean())**2).sum()
        return score

In [102]:
X_train.shape

(67, 99)

In [109]:
NormalEquation().fit(X[:, :-1], target).score(X[:, :-1], target)

1.0

In [107]:
LinearRegression().fit(X_train, y_train).score(X_train, y_train)

1.0

In [42]:
n_observations = range(2_500, 100_001, 2_500)
# n_observations = range(100, 2001, 100)

i = 0

normal_equation = []
sklearn_LinRegr = []
for n in n_observations:
    y, X = create_data(n_feats=8, n_obs=n)
    X = minmax_scale(X)
    
    normal_equation_sub = []
    sklearn_LinRegr_sub = []
    
    for _ in range(5):
    
        start = time()
        NormalEquation().fit(X, y)
        end = time()
        normal_equation_sub.append(end - start)

        start = time()
        LinearRegression().fit(X, y)
        end = time()
        sklearn_LinRegr_sub.append(end - start)
        
    normal_equation.append(sum(normal_equation_sub)/5)
    sklearn_LinRegr.append(sum(sklearn_LinRegr_sub)/5)

    print(i)
    i += 1

0
1
2
3
4
5
6


KeyboardInterrupt: 

In [ ]:
import pandas as pd

x = np.arange(100)
source = pd.DataFrame({
    'Duration': normal_equation + sklearn_LinRegr,
    'Model':['Normal Equation'] * 40 + ['Sklearn'] * 40,
    'N Observations': list(n_observations) * 2
})

alt.Chart(source).mark_line().encode(
    x='N Observations',
    y='Duration',
    color='Model'
)


In [ ]:
n_feats = range(10, 101, 10)

i = 0

normal_equation = []
sklearn_LinRegr = []
for n in n_feats:
    y, X = create_data(n_feats=n, n_obs=10_000)
    X = minmax_scale(X)
    
    normal_equation_sub = []
    sklearn_LinRegr_sub = []
    
    for _ in range(5):
    
        start = time()
        NormalEquation().fit(X, y)
        end = time()
        normal_equation_sub.append(end - start)

        start = time()
        LinearRegression().fit(X, y)
        end = time()
        sklearn_LinRegr_sub.append(end - start)
        
    normal_equation.append(sum(normal_equation_sub)/5)
    sklearn_LinRegr.append(sum(sklearn_LinRegr_sub)/5)

    print(i)
    i += 1

In [ ]:
import pandas as pd

x = np.arange(100)
source = pd.DataFrame({
    'Duration': normal_equation + sklearn_LinRegr,
    'Model':['Normal Equation'] * 10 + ['Sklearn'] * 10,
    'N Features': list(n_feats) * 2
})

alt.Chart(source).mark_line().encode(
    x='N Features',
    y='Duration',
    color='Model'
)


In [77]:
y, X = create_data(n_feats=10_000, n_obs=10_000)

In [78]:
X_scaled = minmax_scale(X)

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y,
                                                    test_size=0.33,
                                                    random_state=42)

In [85]:
ne = NormalEquation()
ne.fit(X_train, y_train)
ne.score(X_test, y_test)

-54643372.58079182

In [89]:
ne.coefs_

array([495727.05459841, 317772.91850209, 274498.81326017, ...,
        56785.84588652,  33635.2016939 ,  73501.73016676])

In [93]:
lr.

True

In [86]:
ne.score(X_train, y_train)

-8981.175180256967

In [95]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [96]:
lr.score(X_train, y_train)

1.0

In [98]:
lr.coef_

array([-0.06580018,  0.192062  ,  0.12335203, ..., -0.10378055,
        0.88402512,  0.25159258])

In [83]:
# %%timeit
LinearRegression().fit(X_train, y_train).score(X_test, y_test)

0.9999448592509455

Note that we create an additional column of all ones in our training data and out test data. This will act as the intercept.  
If you didn't add the column of ones, the equation would calculate all coefficients given an intercept of `0`. 

An exercise and thought experiment for you:  

> Try timing this implementation against sklearn's LinearRegression with varying numbers of features and observations. How does time complexity scale? Why is this the case?

> Think about how one would account for an intercept using the normal equation without adding the column of ones.  <br>
>> Hint: what does the intercept do, and what might you do so that an intercept of `0` is the ideal intercept?

Side note:  
For anymore unfamiliar with the `__repr__` "dunder", try deleting it and finding out what changes. If you'd like to learn more, I strongly recommend reading [Fluent Python](http://index-of.es/Varios-2/Fluent%20Python%20Clear%20Concise%20and%20Effective%20Programming.pdf). In it, they go over `__repr__` and some of the other "magic methods" found in python and how they compare to those found in other programming languages (as well as pretty anything else you might even want to know about python).

## Stochastic Gradient Descent

While the normal equation above is incredibly fast given the size of our data (for those who didn't try on their own, it's actually significantly faster than sklearn for a dataset of this size), it does not scale particularly well; that's why we typically use gradient descent in practice.

Below I'll walk through taking the derivative of the cost function for gradient descent as well as provide some sudo code to warm us up before we dive into any more full fledged models.

I'll walk through this derivative both becuase it is simpler than the derivative of RSS and it's what you'd likely be expected to code if you were asked to create a linear regression model in an interview. Knowing how to derive it on your own should help with remembering it for when it's necessary.

### Differentiating Gradient Descent

We start with a single squared error, or the _definition of the cost function_ which you can see below using two types of notation:  

$ (y - (mx + b))^2 $

and  
$\frac{1}{2m}\sum_{i=1}^{m} \left(h_{\theta}(x^{(i)})-y^{(i)}\right)^2$

We'll use the first notation going forward.

As mentioned, we need to take the derivative of the cost function. We will take it with respect to both $m$ (our coefficient) and $b$ (our intercept).

#### Derivative with respect to $m$:  

$\frac{d}{dm}\left[(y - (mx + b))^2\right]$ ->  

$2 \times (y - (mx + b)) \frac{d}{dm}\left[(y - (mx + b))\right]$ ->  

$\frac{d}{dm} 2 \times (y - (mx + b)) \times x$  

We don't actually need the $2$ (we're minimizing this... it doesn't matter if we minimize $x$ or $2x$) so we just end up with:  

$\frac{d}{dm} (y - (mx + b)) \times x$  

#### Taking steps with $m$:  

We multiply the derivative by the learning rate to get our step.  
The code looks something like this:
``` python
pred = mx + b
error = y - pred
derivative = error * x
step = derivative * learning_rate
```
After taking the derivative with respect to $m$, the derivative with respect to $b$ should be slightly easier:

#### Derivative with respect to $b$:  

$\frac{d}{dm}\left[(y - (mx + b))^2\right]$ ->  

$2 \times (y - (mx + b)) \frac{d}{dm}\left[(y - (mx + b))\right]$ ->  

$\frac{d}{dm} 2 \times (y - (mx + b)) \times 1$ ->

$\frac{d}{dm} y - (mx + b)$

The only difference is that we're not multiplying $b$ by anything so it essentially dissappears after taking its derivative in the second stage of the chain.  

#### Taking steps with $b$:  
Now we take a look at the code (note how similar it is to the step for the coefficient, $m$):
``` python
pred = mx + b
error = y - pred
derivative = error
step = derivative * learning_rate
```

### Combining what we've learned
The step for the intercept ($b$) is just the error multiplied by the learning rate!  
The last two lines of code can obviously be simplified to `step = error * learning_rate`.  

This can be abstracted to matrix multiplication quite simply.  
Using `numpy`, the `@` operator will act as a matrix multiplier (`numpy` also has `arr.dot(arr2)` but I haven't noticed any performance improvments and I find `@` more readable). We simply swap out the `*` with `@` wherever we're multiplying matrices (but don't forget that order matters during matrix multiplication!).  
(_Note that due to matrix multiplication magic, this can work with a single observation, a batch of observations, or even an entire dataset._ You simply need to divide by the number of observations used in the calculation and multiply by the learning rate).
``` python
n = len(X[0])
pred = (X @ m) + b
error = (y - pred) * (1/n)
coefs += error @ x * learning_rate
intercept += error.sum() * learning_rate
```

### But why do we actually need a learning rate?

#### An analogy to get you thinking
Imagine you and 3 friends are trying to lift a table. The table weighs 100 pounds, but at the start each person is lifting only 10 pounds of the table's weight.

That's 40 pounds in total, so everyone combined is 60 pounds short of holding the entire weight of the table; they haven't managed to move/lift it yet.

If one person were to lift the remaining 60 pounds, they might manage to lift the table, but it would be lopsided (or maybe even flip!). We can compare this to modifying a single coefficient without using a learning rate. In this example, that one coefficient (person) would take the entire remaining weight of the table. 

If each one were to independently decide to lift the remaining 60 pounds, and they did so simultaneously, they would likely end up throwing the table into the air! We can compare this to over-fitting like _crazy_. Each coefficient is trying to do all of the work (lift the entire remaining weight of the table) on its own.

The solution is for everyone to constantly make small adjustments until they smoothly/evenly lift the table off of the floor with each person ultimately taking about 25 pounds of the table's weight. 

To further the comparison to coefficients, if one side/corner of the table weighed more, then the person holding that side/corner would need to lift more. Lifting more weight would be like increasing the coefficient for that feature. Pushing this analogy even furhter; when all people are lifting the same amount, it's like having 4 features with perfect multicolinearity. But, unlike a single perfectly weighted coefficient, we can't all lift a 100 pound table on our own.

#### Showing it with data:
Below I train a simple model without using a learning rate, i.e. no small adustments.  
This will result in **_extreme_** over-fitting as each coefficient will try to "explain" all of the change on its own.  
I'll demonstrate this by showing two examples. 

Below, we train our very simple model:

In [44]:
# Array of ones for our coefficients
m = np.zeros_like(X_train[0])
# The intercept
b = 0
# The number of oberservations
n = X_train.shape[0]

pred = X_train @ m + b
error = (2/n) * (y_train - pred)

d_m = error.T @ X_train
d_b = error.sum()

m += d_m
b += d_b

##### First example - Using the Intercept as a predictor:
The predicted `b` is equal to the mean of our target. The best (and only) prediction that can be made when there are no features (e.g. only an intercept and a target).  
Pop-quiz: What will the r-squared be if we make predictions using only the intercept, `b`?

In [45]:
print(b/2, "- The intercept that we've created")
print(y_train.mean(), "- The mean of the target")

984.8134405883508 - The intercept that we've created
984.8134405883509 - The mean of the target


##### Second example - Using only one of our coefficients:
The 8 coefficients without the intercept actually provide a reasonably accurate prediction for our target varianle. It makes sense that this works without the target variable because the adjustments that resulted in these coefficients were made without any knowledge that a y intercept might ever exist. They are the best fit given the available information. 

In [46]:
results = X_test @ m
r2_score(y_test, results/4)

0.9303584913819554

So, that's what happens you don't take steps!

Now let's go build a model that takes steps.

### Implementing a Stochastic Gradient Descent Model
Now we can finally start creating our gradient descent model!

I'm sure you can figure out most of what's going here but I'll point out a few things. We now have `learning_rate` and `max_iter` parameters. These weren't necessary for the normal equation because it's not iterative. Here we're taking steps; we need to know how big they are, and in case we're never going to converge, maybe it's just better to quit early and change your step size (and hopefully that allows you to converge... or just increase the number of iterations).

There are many more parameters that I could add to this model for any number of reasons, but these will do for our purposes.

In [14]:
class StochasticGradientDescent:
    
    def __init__(self, learning_rate=0.05, max_iter=1_000):
        self.l = learning_rate
        self.max_iter = max_iter
        
    def __repr__(self):
        parameters = (self.max_iter, self.l)
        return "LinearRegression(max_iter=%s, learning_rate=%s)" % parameters
    
    def _gradient_descent(self, x, y):
        # Remember that order matters for matrix multiplication but numpy
        # allows some flexibility when multiplying (k, ) matrices which
        # allows us to use 'mx + b' style notation
        pred = self.coefs_ @ x + self.intercept
        error = y - pred
        self.coefs_ += error * x * self.l
        self.intercept += error * self.l
    
    def fit(self, X_train, y_train):
        self.intercept = 0
        
        # Create a numpy array of arbitrarily valued coefficients
        # that will be updated as we iterate
        self.coefs_ = np.ones_like(X_train[0])
        
        # We keep track of our r-squared 
        # When the r-squared in no longer decreasing above a certain
        # rate. We've assume we've converged.
        old_r2 = 0
        
        converged = False
        
        while not converged:
            for row in zip(X_train, y_train):
                
                # Array of feature values for a single observation
                x = row[0]
                
                # Target value for a single observation
                y_obs = row[1]
            
                # Perform gradient descent and update 
                # our coefficients and intercept
                self._gradient_descent(x, y_obs)
            
            # Create predictions
            pred = (X_train @ self.coefs_) + self.intercept
            
            # Calculate the r-squared for those predictions
            new_r2 = r2_score(pred, y_train)
            
            # Check the change in r-squared
            if abs((old_r2) - (new_r2)) < 1e-15:
                converged = True
                
            # Update the old r-squared
            old_r2 = new_r2
        
        return self
        
    def predict(self, X_test):
        preds = (X_test @ self.coefs_) + self.intercept
        return preds
    
    def score(self, X_test, y_test):
        preds = (X_test @ self.coefs_) + self.intercept
        return r2_score(y_test, preds)

In [16]:
StochasticGradientDescent().fit(X_train, y_train).score(X_test, y_test)

0.976366635108409

## Batch Gradient Descent

While stochastic gradient descent gets the job done, it's not the ideal model. Such frequent steps result in noisy updates for the coefficients and features on their way to the minima and convergence typically takessignificantly longer. One way to counteract this is via batch gradient descent. 

In essence, batch gradient descent looks over the entire set of training data and takes the average of the steps indicated. This provides a significantly more robust solution for calculating the step size.

However, batch gradient descent is not without it's shortcominings. It uses the entire set of training data for each step. This becomes quite expensive as the number of iterations increases. 

This problem is solved via mini-bath gradient descent in which a random subset of observations are used for each iteration. Here, I've decided to take 10% of the observations. 

### Mini-Batch Gradient Descent

In [23]:
class MiniBatchGradientDescent:
    
    def __init__(self, max_iter=1_000, learning_rate=.05):
        self.max_iter = max_iter
        self.learning_rate = learning_rate
    
    def __repr__(self):
        parameters = (self.max_iter, self.learning_rate)
        return "LinearRegression(max_iter=%s, learning_rate=%s)" % parameters
    
    def _gradient_descent(self, X, y):
        pred = X @ self.coefs_ + self.intercept
        error = (1/self.batch_size) * (y - pred)

        # error.T = (1 x n) 
        # X = (n x k) 
        # error x k -> (k x 1)
        # These will become our updated coefficients
        d_coefs = error.T @ X
        d_intercept = error.sum()

        self.coefs_ += self.learning_rate * d_coefs
        self.intercept += self.learning_rate * d_intercept
    
    def fit(self, X_train, y_train):
        self.coefs_ = np.zeros_like(X_train[0])
        self.intercept = 0
        
        # Our number of observations
        n = X_train.shape[0]
        
        # Calculating batch size from number of observations
        self.batch_size = int(n/10)
        
        # Keeping track of number of iterations
        i = 0
        
        old_r2 = 0
        converged = False

        while not converged:
            batch_indices = np.random.choice(n, self.batch_size)
            X_batch = X_train[batch_indices, :]
            y_batch = y_train[batch_indices]
            
            # Perform gradient descent and update 
            # our coefficients and intercept
            self._gradient_descent(X_batch, y_batch)

            pred = (X_train @ self.coefs_) + self.intercept
            new_r2 = r2_score(pred, y_train)
            
            i += 1
            if abs((old_r2) - (new_r2)) < 1e-5:
                converged = True
            if i >= self.max_iter:
                return self, print("Failed to converged after", i, "iterations.")
            old_r2 = new_r2
            
        return self
        
    def predict(self, X_test):
        preds = (X_test @ self.coefs_) + self.intercept
        return preds
    
    def score(self, X_test, y_test):
        preds = (X_test @ self.coefs_) + self.intercept
        return r2_score(y_test, preds)

In [31]:
MiniBatchGradientDescent(max_iter=10_000).fit(X_train, y_train).score(X_test, y_test)

0.9538424013855976

Not as fast or as reliable as sklearn's model, but an understanding of what is happening in the above will be a great asset if you ever need to understand how linear regression works under the hood.

I hope you've enjoyed reading this as much as I went crazy writing it!